# Kaskadiertes Question Answering

Wie du im letzten Teil gesehen hast, funktioniert das automatische Beantworten von Fragen schon sehr gut, wenn sich die entsprechende Antwort im Dokument befindet.

Leider hast du es oft nicht nur mit einem einzelnen Dokument zu tun, sondern mit sehr vielen. Da sieht die Sache etwas schwieriger aus, denn du musst erst das richtige Dokument finden, das potenziell die Antwort auf deine Frage enthält.

Dazu benötigst du ein *kaskadiertes Modell*, das dir aus einer großen Menge von Dokumenten erst das richtige heraussucht. Mit den Techniken, die du bisher gelernt hast, könntest du das selbst herausfinden, z.B. mithilfe der semantischen Transformation. Es gibt allerdings auch schon fertig Frameworks, die das für dich erledigen. Ein solches wirst du dir jetzt genauer anschauen.

## Daten einladen

Wie beim Transfer Learning gewohnt lädst du die Original-Daten ein:

In [ ]:
import os
os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
newsticker_db = 'heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, datePublished, url, full_text FROM nlp_articles WHERE datePublished<'2021-01-01' ", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Nutzung des Haystack-Frameworks

Um dir Programmieraufwand für die Selektion der richtigen Daten zu sparen, kannst du das [Haystack-Framework](https://haystack.deepset.ai) verwenden. Es ist relativ umfangreich, du benötigst allerdings nur die Teile, die dir die Dokumente heraussuchen und dann Fragen dazu beantworten.

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install urllib3==1.25.4

In [ ]:
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

In [ ]:
from haystack.document_store.memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

Die Dokumente müssen dafür ein eine Form konvertiert werden, in der neben dem reinen Text auch noch Metainformationen gespeichert sind:

In [ ]:
dicts = [{"content": row["full_text"], "meta": {"name": row["url"]}} for i, row in df.iterrows()]

Anschließend kannst du die Dokumente abspeichern, dabei passiert allerdings noch ziemlich wenig:

In [ ]:
document_store.write_documents(dicts)

Erst wenn du das erste Mal auf die Daten zugreifst, werden diese gewandelt. Sehr praktisch daran ist, dass einzelne Paragraphen separat behandelt werden. Oft ist der Zusammenhang ja auch dadurch gegeben!

In [ ]:
from haystack.retriever.sparse import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

In diesem Beispiel verwendest du entweder das von Deepset empfohlene Modell oder das aus der vorherigen Lektion:

In [ ]:
#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
reader = FARMReader(model_name_or_path="Sahajtomar/German-question-answer-Electra", use_gpu=True)

Eine *Pipeline* besteht hier aus einem Reader und einem Retriever. Es werden also zuerst die passenden Dokumente gefunden und anschließend wird versucht, mithilfe dieser Dokumente die Frage zu beantworten:

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

Nun kannst du Fragen an den gesamten Heise Newsticker des Jahres 2020 stellen:

In [ ]:
prediction = pipe.run(query="Wer ist der Chef von Apple?")
print_answers(prediction, details="minimal")

Das hat ganz gut geklappt, versuche es nun mit einer etwas spezielleren Frage:

In [ ]:
prediction = pipe.run(query="Wie viel verdient Tim Cook?") 
print_answers(prediction, details="minimal")

Leider nicht ganz die passende Antwort. Wechsle daher den Konzern und stelle eine Frage zu Google:

In [ ]:
prediction = pipe.run(query="Wie hoch ist der Gewinn von Google?")
print_answers(prediction, details="minimal")

Die Richtung stimmt, aber das passt auch nicht ganz.

Vielleicht klappt es besser mit allgemeinen Begriffen?

In [ ]:
prediction = pipe.run(query="Was ist Corona?")
print_answers(prediction, details="minimal")

Das ist schon ziemlich gut!

Starte noch einen Versuch:

In [ ]:
prediction = pipe.run(query="Was ist das wichtigste Produkt von Microsoft?")
print_answers(prediction, details="minimal")

Man könnte fast meinen, das System merkt sich die Fragen. *Teams* ist vielleicht aktuell wirklich für viele Menschen das wichtigste Produkt von Microsoft!

## Fragen aus vielen Dokumenten zu beantworten ist schwierig

Nicht umsonst haben sich Chatbots noch nicht so richtig durchgesetzt. Wie du siehst, ist es nicht so einfach, aus einer großen Dokumentemenge das richtige auszuwählen und Fragen korrekt zu beantworten.

Das liegt daran, dass sich hier zwei schwierige Probleme miteinander verbinden:
* Information Retrieval
* Question Answering

In beiden Bereichen gibt es aktuell große Fortschritte. In vielleicht sogar kurzer Zeit kannst du solche Anforderungen daher möglicherweise bereits viel besser lösen.